In [ ]:
# 班級：新聞三乙 姓名：胡靜珊  學號：A105010086 
# A. Regression
#from sklearn.tree import DecisionTreeRegressor
#tree_reg = DecisionTreeRegressor(max_leaf_nodes=100,min_samples_split=3,min_samples_leaf=4,presort=False,min_impurity_decrease=8.2)
#tree_reg.fit(X_train, y_train) 
#39282.25319504781


# B. Classification 
#from sklearn.ensemble import AdaBoostClassifier
#ada_clf = AdaBoostClassifier(
#DecisionTreeClassifier(max_depth=2), n_estimators=150,
#algorithm="SAMME.R", learning_rate=0.5
#)
#ada_clf.fit(X_train, y_train)
#0.8866279069767442

In [2]:
import pandas as pd
#import此package將他簡稱為pd (panda為python的excel,把資料讀出來比較好看)
import numpy as np
#將numpy簡稱為np(numpy矩陣的運算)

#### Quick look at the data structure

In [3]:
housing = pd.read_csv("housing.csv")
#pandas將"housing.csv"這個檔讀進來,assign(=)為housing ,housing現在就是讀進來的這個檔

In [4]:
housing.head()
#檢查housing 這個檔長什麼樣子,head()是放前5筆,()中若填數字就能決定可show幾筆
#此是要預測median_house_value,所以在這個檔裡,y=median_house_value,剩下的都是x

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
housing.info()
#看現在這個csv(housing)的基本資料
#float小數   ocean_proximity  object-是文字(NEAR BAY)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
housing["ocean_proximity"].value_counts()
#要看housing中的某個屬性的內容 .value_counts()-去計算每個類型有多少(每一個分類的樣本數有多少)

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [7]:
print(housing.describe())

          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.000000   
25%     -121.800000     33.930000           18.000000   1447.750000   
50%     -118.490000     34.260000           29.000000   2127.000000   
75%     -118.010000     37.710000           37.000000   3148.000000   
max     -114.310000     41.950000           52.000000  39320.000000   

       total_bedrooms    population    households  median_income  \
count    20433.000000  20640.000000  20640.000000   20640.000000   
mean       537.870553   1425.476744    499.539680       3.870671   
std        421.385070   1132.462122    382.329753       1.899822   
min          1.000000      3.000000      1.000000       0.499900   
25%        296.00000

#### Looking for correlation

In [8]:
#correlation-相關係數
corr_matrix = housing.corr()
#show出相關係數(會跑出9*9的矩陣,要先寫此步讓他知housing.corr()assign為corr_matrix
corr_matrix["median_house_value"].sort_values(ascending = False)
#對median_house_value,show出"median_house_value"的相關係數((因今要預測出"median_house_value"

median_house_value    1.000000
median_income         0.688075
total_rooms           0.134153
housing_median_age    0.105623
households            0.065843
total_bedrooms        0.049686
population           -0.024650
longitude            -0.045967
latitude             -0.144160
Name: median_house_value, dtype: float64

In [9]:
#上面結果的屬性是原本就有的(他給的),可透過下面的方式新增屬性
housing["rooms_per_household"] = housing["total_rooms"] / housing["population"]
housing["population_per_household"] = housing["population"] / housing["households"]
#再透過下面方式將"median_house_value"的相關係數show出來
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value          1.000000
median_income               0.688075
rooms_per_household         0.209482
total_rooms                 0.134153
housing_median_age          0.105623
households                  0.065843
total_bedrooms              0.049686
population_per_household   -0.023737
population                 -0.024650
longitude                  -0.045967
latitude                   -0.144160
Name: median_house_value, dtype: float64

#### Prepare the data for Machine Learning algorithms

In [10]:
#數據分析,第一步是要先把null值(空值,缺失值)處理好
housing[housing.isnull().any(axis=1)].head(10)
#判斷在任何一個row裡面(any(axis=1)),是不是有null值  housing的任何一個軸是不是有null
#有的話先把前十筆秀出來
#下表結果NaN 就是null值 (not a number

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_per_household
290,-122.16,37.77,47.0,1256.0,NaN,570.0,218.0,4.3750,161900.0,NEAR BAY,2.203509,2.614679
341,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,NEAR BAY,1.355191,2.826255
538,-122.28,37.78,29.0,5154.0,NaN,3741.0,1273.0,2.5762,173400.0,NEAR BAY,1.377706,2.938727
563,-122.24,37.75,45.0,891.0,NaN,384.0,146.0,4.9489,247100.0,NEAR BAY,2.320312,2.630137
696,-122.10,37.69,41.0,746.0,NaN,387.0,161.0,3.9063,178400.0,NEAR BAY,1.927649,2.403727
738,-122.14,37.67,37.0,3342.0,NaN,1635.0,557.0,4.7933,186900.0,NEAR BAY,2.044037,2.935368
1097,-121.77,39.66,20.0,3759.0,NaN,1705.0,600.0,4.7120,158600.0,INLAND,2.204692,2.841667
1350,-121.95,38.03,5.0,5526.0,NaN,3207.0,1012.0,4.0767,143100.0,INLAND,1.723106,3.168972
1456,-121.98,37.96,22.0,2987.0,NaN,1420.0,540.0,3.6500,204100.0,INLAND,2.103521,2.629630
1493,-122.01,37.94,23.0,3741.0,NaN,1339.0,499.0,6.7061,322300.0,NEAR BAY,2.793876,2.683367


In [11]:
#p.60
#在處理nan時有3種作法
# housing = housing.dropna(subset=["total_bedrooms"])    # option 1
#把樣本-(total_bedrooms是nan)就把他給拿掉
housing = housing.drop("total_bedrooms", axis=1)       # option 2
#因由上一步的步驟結果發現有空值的都是total_bedrooms,所以直接把total_bedrooms這個屬性給拿掉,下執行此方式,total_bedrooms消失
#透過drop的方式把它拿掉

# median = housing["total_bedrooms"].median()            # option 3
# housing["total_bedrooms"].fillna(median, inplace=True) # option 3
housing

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_per_household
0,-122.23,37.88,41.0,880.0,322.0,126.0,8.3252,452600.0,NEAR BAY,2.732919,2.555556
1,-122.22,37.86,21.0,7099.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,2.956685,2.109842
2,-122.24,37.85,52.0,1467.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2.957661,2.802260
3,-122.25,37.85,52.0,1274.0,558.0,219.0,5.6431,341300.0,NEAR BAY,2.283154,2.547945
4,-122.25,37.85,52.0,1627.0,565.0,259.0,3.8462,342200.0,NEAR BAY,2.879646,2.181467
5,-122.25,37.85,52.0,919.0,413.0,193.0,4.0368,269700.0,NEAR BAY,2.225182,2.139896
6,-122.25,37.84,52.0,2535.0,1094.0,514.0,3.6591,299200.0,NEAR BAY,2.317185,2.128405
7,-122.25,37.84,52.0,3104.0,1157.0,647.0,3.1200,241400.0,NEAR BAY,2.682800,1.788253
8,-122.26,37.84,42.0,2555.0,1206.0,595.0,2.0804,226700.0,NEAR BAY,2.118574,2.026891
9,-122.25,37.84,52.0,3549.0,1551.0,714.0,3.6912,261100.0,NEAR BAY,2.288201,2.172269


#### Handling Text and Categorical Attributes

In [12]:
type(housing[["longitude"]])

pandas.core.frame.DataFrame

In [13]:
type(housing["longitude"])

pandas.core.series.Series

In [14]:
housing_cat = housing[["ocean_proximity"]]
# housing_cat cat-(category) 
#把"ocean_proximity" 原是timeseries,變成DataFrame
housing_cat.head(10)

,ocean_proximity
0,NEAR BAY
1,NEAR BAY
2,NEAR BAY
3,NEAR BAY
4,NEAR BAY
5,NEAR BAY
6,NEAR BAY
7,NEAR BAY
8,NEAR BAY
9,NEAR BAY


In [15]:
#p.63
# 分類作法 A
#處理文字的部分,從sklearn import preprocessing(前處理)這個package
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
#creat一個東西叫做LabelEncoder,叫做encoder
housing_cat = housing["ocean_proximity"]
#將該 housing內的"ocean_proximity"資料assign給housing_cat  現在輸入housing_cat 會跑出housing["ocean_proximity"]的資料
housing_cat_encoded = encoder.fit_transform(housing_cat)
#encoder就是LabelEncoder,用LabelEncoder去fit跟transform///fit-讓他根據資料的型態去找資料的樣子 transform-找完之後去做這件事情 ()內放要處理的屬性
housing_cat_encoded
#所以用完之後會發現housing_cat_encoded變成333....111((可知成功將文字(NEAR BAY...)轉換成數字

#利用 LabelEncoder將文字轉換成數字((將文字用樹來表示

array([3, 3, 3, ..., 1, 1, 1])

In [16]:
print(encoder.classes_)
#編碼有以下幾種情形,0 1 2 3 4 ((python從0開始編碼

['<1H OCEAN' 'INLAND' 'ISLAND' 'NEAR BAY' 'NEAR OCEAN']


In [17]:
#p.63
#5/3-1343-05:16
#要把[3, 3, 3, ..., 1, 1, 1]這種數字變成dummy(就是0,1的型態)
#OneHotEncoder就是把12345這種數字換成是dummy,來處理數字

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(categories='auto')
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
#因原本housing_cat_encoded.shape是(20640,)//只有一個維度,標準的vactor(可參照github-shu_test.ipynb步驟23的輸出結果),後面是空的,今天
#這裡要求的資料要是二維的
#要進入OneHotEncoder的資料,形狀要有指定,後面那個維度要補上,所以reshape成(-1,1)的形式 可參照github-shu_test.ipynb步驟24-25
#(-1.1)是一個固定的用法 
#讓他變成後面那個維度就只有1
housing_cat_1hot

#執行完之後變成sparse matrix每一個row(橫的)都只有一個一,其他都是0 e.x.[0., 0., 0., 1., 0.],[0., 0., 0., 1., 0.]

<20640x5 sparse matrix of type '<class 'numpy.float64'>'
	with 20640 stored elements in Compressed Sparse Row format>

In [18]:
housing_cat_1hot.toarray()

#好了之後要利用此矩陣來取代原始的ocean_proximity,把ocean_proximity這些值換成一組有意義的數字

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [19]:
#p.64
# 分類作法 B
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
encoder.fit_transform(housing_cat)

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

#### combine housing and categorical 

In [20]:
pd.DataFrame(housing_cat_1hot.toarray()).iloc[:, 1:].head()

,1,2,3,4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [21]:
housing_final = pd.concat([housing, pd.DataFrame(housing_cat_1hot.toarray()).iloc[:, 1:]], axis=1)
#concat將,前後兩個的連在一起
#iloc將第一個得拿掉,:代表是所有的row,1-colum是從1開始//因為5個分類只要4個dummy EX.要分兩類,用一個變數將其分開,要分3類,用2個變數將其分開(以此類推)

In [22]:
housing_final.head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_per_household,1,2,3,4
0,-122.23,37.88,41.0,880.0,322.0,126.0,8.3252,452600.0,NEAR BAY,2.732919,2.555556,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,2.956685,2.109842,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2.957661,2.802260,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,558.0,219.0,5.6431,341300.0,NEAR BAY,2.283154,2.547945,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,565.0,259.0,3.8462,342200.0,NEAR BAY,2.879646,2.181467,0.0,0.0,1.0,0.0


# Prepare the data for Machine Learning algorithms

In [23]:
X = housing_final.drop("median_house_value", axis=1).drop("ocean_proximity", axis=1)
X.head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,rooms_per_household,population_per_household,1,2,3,4
0,-122.23,37.88,41.0,880.0,322.0,126.0,8.3252,2.732919,2.555556,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,2401.0,1138.0,8.3014,2.956685,2.109842,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,496.0,177.0,7.2574,2.957661,2.802260,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,558.0,219.0,5.6431,2.283154,2.547945,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,565.0,259.0,3.8462,2.879646,2.181467,0.0,0.0,1.0,0.0


In [24]:
y = housing_final[["median_house_value"]]
y.head()

,median_house_value
0,452600.0
1,358500.0
2,352100.0
3,341300.0
4,342200.0


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
# sc_X.fit_transform(X_train["longitude"])

In [27]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,rooms_per_household,population_per_household,1,2,3,4
12069,-117.55,33.83,6.0,502.0,228.0,65.0,4.2386,2.201754,3.507692,1.0,0.0,0.0,0.0
15925,-122.44,37.73,52.0,2381.0,1485.0,447.0,4.3898,1.603367,3.322148,0.0,0.0,1.0,0.0
11162,-118.00,33.83,26.0,1718.0,1022.0,368.0,3.9333,1.681018,2.777174,0.0,0.0,0.0,0.0
4904,-118.26,34.01,38.0,697.0,749.0,206.0,1.4653,0.930574,3.635922,0.0,0.0,0.0,0.0
4683,-118.36,34.08,52.0,2373.0,1135.0,576.0,3.1765,2.090749,1.970486,0.0,0.0,0.0,0.0


In [28]:
X_train[['longitude', 'latitude']] = sc_X.fit_transform(X_train[['longitude', 'latitude']])

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\pandas\core\frame.py:3395: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\pandas\core\frame.py:3367: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pa

In [29]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,rooms_per_household,population_per_household,1,2,3,4
12069,1.003899,-0.840062,6.0,502.0,228.0,65.0,4.2386,2.201754,3.507692,1.0,0.0,0.0,0.0
15925,-1.434772,0.985364,52.0,2381.0,1485.0,447.0,4.3898,1.603367,3.322148,0.0,0.0,1.0,0.0
11162,0.779481,-0.840062,26.0,1718.0,1022.0,368.0,3.9333,1.681018,2.777174,0.0,0.0,0.0,0.0
4904,0.649818,-0.755812,38.0,697.0,749.0,206.0,1.4653,0.930574,3.635922,0.0,0.0,0.0,0.0
4683,0.599947,-0.723048,52.0,2373.0,1135.0,576.0,3.1765,2.090749,1.970486,0.0,0.0,0.0,0.0


In [30]:
# # Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)
# sc_y = StandardScaler()
# y_train = sc_y.fit_transform(y_train)

In [31]:
type(y_train)

pandas.core.frame.DataFrame

In [32]:
type(X_train)

pandas.core.frame.DataFrame

#### null detection

In [33]:
# 方法 A
# y_train.isnull().any(axis=1)
# X_train.isnull().any(axis=1)

# 方法 B
# np.isnan(y_train).any()
# np.isnan(X_train).any()

# Prepare the data for Machine Learning algorithms

In [34]:
#線性回歸模型
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
# lin_reg.predict(X_train)
# lin_reg.predict(X_test)

#### 檢驗 train set 績效

In [36]:
#p.71
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

67973.296447151
49461.47745791854


#### 檢驗 test set 績效

In [37]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

4959090.752590909
4958542.753705764


#### 模型換決策樹

In [38]:
#p.71
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
# housing_predictions = tree_reg.predict(housing_prepared)
# tree_mse = mean_squared_error(housing_labels, housing_predictions)
# tree_rmse = np.sqrt(tree_mse)
# tree_rmse

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

#### 檢驗 train set的績效

In [39]:
from sklearn.metrics import mean_squared_error

housing_predictions = tree_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

0.0
0.0


#### 檢驗 test set 績效

In [40]:
from sklearn.metrics import mean_squared_error

housing_predictions = tree_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

102897.58825261386
76078.6453488372


# Fine-tune your model

In [41]:
#P.72
from sklearn.model_selection import cross_val_score

tree_scores = cross_val_score(tree_reg, X_train, y_train,
                              scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)

In [42]:
tree_rmse_scores

array([73772.76667457, 67063.34136115, 68486.01118806, 69235.0758387 ,
       74758.36433447, 72861.1333433 , 66106.04076571, 65866.66993   ,
       69546.84750217, 63734.15731391])

In [43]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [73772.76667457 67063.34136115 68486.01118806 69235.0758387
 74758.36433447 72861.1333433  66106.04076571 65866.66993
 69546.84750217 63734.15731391]
Mean: 69143.04082520421
Standard deviation: 3477.5393031734807


In [44]:
lin_scores = cross_val_score(lin_reg, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [68945.97646887 65590.9449141  69121.65720735 67253.67505489
 72308.16414768 67532.58484636 66712.83282223 68798.4654893
 71303.52108227 64654.42581677]
Mean: 68222.22478498152
Standard deviation: 2262.7177251532084


In [45]:
#.73
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)
housing_predictions = forest_reg.predict(X_train)
forest_mse = mean_squared_error(y_train, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


21691.589321524494

In [46]:
#p.72
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, X_train, y_train,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

Scores: [55572.18098362 50831.05455199 51381.92604848 51213.8585785
 55134.92182812 52479.2018058  50688.2990192  52208.03479914
 53100.64531123 47790.25122696]
Mean: 52040.037415303246
Standard deviation: 2141.6238290398223


In [47]:
scores = cross_val_score(lin_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

count       10.000000
mean     68222.224785
std       2385.113905
min      64654.425817
25%      66848.043380
50%      68165.525168
75%      69077.737023
max      72308.164148
dtype: float64

In [48]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(X_train, y_train)
housing_predictions = svm_reg.predict(X_train)
svm_mse = mean_squared_error(y_train, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


94179.55824305126

In [49]:
#p.74
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
        {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    ]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [2, 4, 6, 8],


In [50]:
#p.74
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [51]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [52]:
#p.75
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

62834.62971816074 {'max_features': 2, 'n_estimators': 3}
54085.90912209878 {'max_features': 2, 'n_estimators': 10}
51414.07166252632 {'max_features': 2, 'n_estimators': 30}
58100.95758635396 {'max_features': 4, 'n_estimators': 3}
51863.20456822892 {'max_features': 4, 'n_estimators': 10}
49700.4446545128 {'max_features': 4, 'n_estimators': 30}
58228.87969574965 {'max_features': 6, 'n_estimators': 3}
51631.109681861795 {'max_features': 6, 'n_estimators': 10}
49274.74815788537 {'max_features': 6, 'n_estimators': 30}
58179.50916519872 {'max_features': 8, 'n_estimators': 3}
51811.46923987178 {'max_features': 8, 'n_estimators': 10}
49754.8568633981 {'max_features': 8, 'n_estimators': 30}
60929.55815684713 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
52505.17205542074 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
58260.73597743199 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
51349.51231035405 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}

In [53]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.093621,0.015075,0.005861,0.007425,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-3.925040e+09,-3.722267e+09,-4.268532e+09,-3.832455e+09,-3.992735e+09,-3.948191e+09,1.841574e+08,18
1,0.291195,0.006971,0.012304,0.004877,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-2.898150e+09,-2.789270e+09,-3.094612e+09,-2.914731e+09,-2.929714e+09,-2.925286e+09,9.801949e+07,11
2,1.075477,0.101488,0.044070,0.009949,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-2.724903e+09,-2.532022e+09,-2.820548e+09,-2.563272e+09,-2.576298e+09,-2.643407e+09,1.107708e+08,6
3,0.153158,0.019794,0.003122,0.002846,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-3.281765e+09,-3.267458e+09,-3.669989e+09,-3.262851e+09,-3.396604e+09,-3.375721e+09,1.551190e+08,13
4,0.492268,0.036801,0.014148,0.002260,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-2.684663e+09,-2.589039e+09,-2.841166e+09,-2.723459e+09,-2.610664e+09,-2.689792e+09,8.998281e+07,9
5,1.344512,0.016905,0.031493,0.000487,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-2.577707e+09,-2.391091e+09,-2.617549e+09,-2.364407e+09,-2.399909e+09,-2.470134e+09,1.055079e+08,2
6,0.188796,0.004280,0.004945,0.006398,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-3.626581e+09,-3.539957e+09,-3.299166e+09,-3.039306e+09,-3.447886e+09,-3.390602e+09,2.064178e+08,15
7,0.627427,0.035745,0.012271,0.002329,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-2.697560e+09,-2.525768e+09,-2.891500e+09,-2.602078e+09,-2.611985e+09,-2.665771e+09,1.253089e+08,7
8,1.856424,0.059335,0.031986,0.001864,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-2.510717e+09,-2.304924e+09,-2.585172e+09,-2.386133e+09,-2.353071e+09,-2.428001e+09,1.039808e+08,1
9,0.252123,0.012643,0.007368,0.003128,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",-3.442344e+09,-3.358324e+09,-3.572412e+09,-3.272993e+09,-3.278195e+09,-3.384855e+09,1.123116e+08,14


In [54]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=10),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor()
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error')
rnd_search.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\model_selection\_validation.py:

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [55]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

52332.98232192202 {'max_features': 3, 'n_estimators': 9}
55687.75921345782 {'max_features': 2, 'n_estimators': 7}
62154.43244839652 {'max_features': 6, 'n_estimators': 2}
53365.26887384945 {'max_features': 7, 'n_estimators': 6}
59035.582192402086 {'max_features': 1, 'n_estimators': 7}
59894.81024213806 {'max_features': 3, 'n_estimators': 3}
51636.88272443597 {'max_features': 7, 'n_estimators': 8}
51728.807358199694 {'max_features': 6, 'n_estimators': 9}
56962.392684878134 {'max_features': 2, 'n_estimators': 6}
59718.02476787068 {'max_features': 2, 'n_estimators': 4}


In [56]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([8.44783193e-02, 8.29890067e-02, 4.28223096e-02, 1.91981237e-02,
       1.96197977e-02, 1.96804015e-02, 3.35517515e-01, 1.38750281e-01,
       7.40450407e-02, 1.76288231e-01, 9.98342067e-05, 2.73577811e-03,
       3.77536247e-03])

In [61]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.5)
lasso_reg.fit(X_train,y_train)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [62]:
from sklearn.metrics import mean_squared_error

housing_predictions = lasso_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

67973.30666357042
49461.68544434042


In [63]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=500.64,l1_ratio=0,tol = 0.00002)
elastic_net.fit(X_train,y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 99340941789791.2, tolerance: 4419838042.92216
  positive)


ElasticNet(alpha=500.64, copy_X=True, fit_intercept=True, l1_ratio=0,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=2e-05, warm_start=False)

In [64]:
from sklearn.metrics import mean_squared_error

housing_predictions = elastic_net.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

109206.75162921014
85372.251073802


In [65]:
from sklearn.metrics import mean_squared_error

housing_predictions = elastic_net.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

107868.4907787101
84363.86149662489


In [66]:
from sklearn.svm import LinearSVR
svm_reg = LinearSVR(epsilon=0.2,C=0.005,fit_intercept=False,random_state=None, intercept_scaling=2.3, dual=True, max_iter=10)
svm_reg.fit(X_train,y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=0.005, dual=True, epsilon=0.2, fit_intercept=False,
          intercept_scaling=2.3, loss='epsilon_insensitive', max_iter=10,
          random_state=None, tol=0.0001, verbose=0)

In [67]:
from sklearn.metrics import mean_squared_error

housing_predictions = svm_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

151026.03808144876
107276.87890686061


In [68]:
from sklearn.metrics import mean_squared_error

housing_predictions = svm_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

148975.485113608
104595.06016343462


In [87]:
from sklearn.svm import SVR
svm_poly_reg = SVR(kernel="precomputed",degree=20,C=5.8,epsilon=2,coef0=10)
# svm_poly_reg.fit(X_train,y_train)

In [88]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_leaf_nodes=100,min_samples_split=3,min_samples_leaf=4,presort=False,min_impurity_decrease=8.2)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=100, min_impurity_decrease=8.2,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [89]:
from sklearn.metrics import mean_squared_error

housing_predictions = tree_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

78702.74147961002
53187.881083823755


In [90]:
from sklearn.metrics import mean_squared_error

housing_predictions =tree_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

55659.236315756905
39282.25319504781


In [92]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=50, warm_start=True)
gbrt.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=50,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=True)

In [93]:
from sklearn.metrics import mean_squared_error

housing_predictions = gbrt.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

3.072835305146626
2.4271182230769504


In [94]:
from sklearn.metrics import mean_squared_error

housing_predictions =gbrt.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

100855.84330932162
74761.68708501609


In [95]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=4, include_bias=False)
X_poly = poly_features.fit_transform(X_train)

In [96]:
X_poly.shape

(16512, 2379)

In [97]:
lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_poly, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [98]:
housing_predictions = lin_reg.predict(X_poly)

In [99]:
housing_predictions.shape
y_train.shape

(16512, 1)

In [100]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(X_poly)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(lin_mae)

47104.03504015889
32492.746562268316


In [101]:
X_test.shape

(4128, 13)

In [102]:
X_poly_test = poly_features.fit_transform(X_test)

In [103]:
X_poly_test.shape

(4128, 2379)

In [104]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(X_poly_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

9.273205393478876e+21
2.0411411412576443e+20


In [ ]:
###shu_test.ipynb-35開始

In [78]:
#把y做出來   ///0510-0138-聽0050之後的音
y = housing_final[["median_house_value"]]  #把housing_final裡面的median_house_value抓出來
y.head()

,median_house_value
0,452600.0
1,358500.0
2,352100.0
3,341300.0
4,342200.0


In [79]:
#把資料做切割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#首先要把X.Y放進去(X, y, test_size = 0.2, random_state = 0)//test_size設20%   ///random_state = 0 代表是兩個所切割的內容及方式是相同的
#前面會產生X_train, X_test, y_train, y_test///
#train_test_split函数用于将矩阵随机划分为训练子集和测试子集，并返回划分好的训练集测试集样本和训练集测试集标签
#train的目的是要fit modle,test是要預測績效

In [80]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
#*************

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [81]:
lin_reg.predict(X_train)
#為下步驟的講解

array([[155784.0103781 ],
       [270001.68948027],
       [220102.94585628],
       ...,
       [258066.28474723],
       [310156.30175896],
       [ 21414.31364214]])

In [82]:
#檢測train set績效 

from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(X_train)
##這個的意思是說:把X_train放進去的時候跑出來的y是什麼(預測出來的y是什麼樣子)
#跑出來的是一組數字,做出來的是X_train所預測的y 
#把此預測出來的y先放到(丟到)housing_predictions
lin_mse = mean_squared_error(y_train, housing_predictions)
#(y_train, housing_predictions)--所預測的y(housing_predictions)跟實際的y(y_train),他們之間的 mean_squared_error
lin_rmse = np.sqrt(lin_mse)
#np.sqrt(x): 平方根
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_train, housing_predictions)
#以此類推做mae的
print(lin_mae)

#//預測績效看兩點間的距離-有兩種方法,一個是平方,一個是取絕對值//通常是看mean_squared_error

67973.29644715102
49461.47745791889


In [83]:
#檢測test set績效
#p.71
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(X_test)
lin_mse = mean_squared_error(y_test, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(y_test, housing_predictions)
print(lin_mae)

68687.96499215413
49435.72918135477


In [84]:
#將課本及此連結在一起
# p. 112/p.70
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [85]:
import copy

In [86]:
housing_classify = copy.deepcopy(housing_final)
housing_classify.head(10)

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_per_household,1,2,3,4
0,-122.23,37.88,41.0,880.0,322.0,126.0,8.3252,452600.0,NEAR BAY,2.732919,2.555556,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,2.956685,2.109842,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2.957661,2.802260,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,558.0,219.0,5.6431,341300.0,NEAR BAY,2.283154,2.547945,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,565.0,259.0,3.8462,342200.0,NEAR BAY,2.879646,2.181467,0.0,0.0,1.0,0.0
5,-122.25,37.85,52.0,919.0,413.0,193.0,4.0368,269700.0,NEAR BAY,2.225182,2.139896,0.0,0.0,1.0,0.0
6,-122.25,37.84,52.0,2535.0,1094.0,514.0,3.6591,299200.0,NEAR BAY,2.317185,2.128405,0.0,0.0,1.0,0.0
7,-122.25,37.84,52.0,3104.0,1157.0,647.0,3.1200,241400.0,NEAR BAY,2.682800,1.788253,0.0,0.0,1.0,0.0
8,-122.26,37.84,42.0,2555.0,1206.0,595.0,2.0804,226700.0,NEAR BAY,2.118574,2.026891,0.0,0.0,1.0,0.0
9,-122.25,37.84,52.0,3549.0,1551.0,714.0,3.6912,261100.0,NEAR BAY,2.288201,2.172269,0.0,0.0,1.0,0.0


In [87]:
housing_classify["median_house_value"].median()

179700.0

In [88]:
housing_classify['median_classify'] = housing_classify["median_house_value"]> housing_classify["median_house_value"].median()
housing_classify['median_classify'].head(10)

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: median_classify, dtype: bool

In [89]:
housing_classify['median_classify'] = housing_classify['median_classify'].astype(int)
housing_classify['median_classify'].head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: median_classify, dtype: int32

In [90]:
X = housing_classify.drop("median_house_value", axis=1).drop("ocean_proximity", axis=1).drop("median_classify", axis=1)

In [91]:
X.head()

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,rooms_per_household,population_per_household,1,2,3,4
0,-122.23,37.88,41.0,880.0,322.0,126.0,8.3252,2.732919,2.555556,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,2401.0,1138.0,8.3014,2.956685,2.109842,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,496.0,177.0,7.2574,2.957661,2.802260,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,558.0,219.0,5.6431,2.283154,2.547945,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,565.0,259.0,3.8462,2.879646,2.181467,0.0,0.0,1.0,0.0


In [92]:
y = housing_classify[["median_classify"]]
y.head()

,median_classify
0,1
1,1
2,1
3,1
4,1


In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [94]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
# train performance
y_pred = log_reg.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = log_reg.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.8277010658914729
0.8241279069767442


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [96]:
softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
softmax_reg.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [97]:
# train performance
y_pred = softmax_reg.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = softmax_reg.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.8160125968992248
0.813468992248062


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [98]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [99]:
svm_clf = Pipeline([
 ("scaler", StandardScaler()),
 ("linear_svc", LinearSVC(C=1, loss="hinge")),
 ])
svm_clf.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [100]:
# train performance
y_pred = svm_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = svm_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.8430232558139535
0.8374515503875969


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [101]:
#p134
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
polynomial_svm_clf = Pipeline([
 ("poly_features", PolynomialFeatures(degree=3)),
 ("scaler", StandardScaler()),
 ("svm_clf", LinearSVC(C=10, loss="hinge"))
 ])
polynomial_svm_clf.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [102]:
# train performance
y_pred = polynomial_svm_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = polynomial_svm_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.828125
0.8108042635658915


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [103]:
#p.135
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
 ("scaler", StandardScaler()),
 ("svm_cif", SVC(kernel = "poly", degree=3, coef0=1, C=5))
 ])
poly_kernel_svm_clf.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_cif',
                 SVC(C=5, cache_size=200, class_weight=None, coef0=1,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [104]:
# train performance
y_pred = poly_kernel_svm_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = poly_kernel_svm_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8816012596899225
0.8657945736434108


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [105]:
#p136
from sklearn.svm import LinearSVR
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [106]:
# train performance
y_pred = svm_reg.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = svm_reg.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.49836482558139533
0.5072674418604651


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [107]:
#p.148
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [108]:
iris = load_iris()
X = iris.data[:, 2:]#petal length and width
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [109]:
# train performance
y_pred = tree_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred = tree_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.7474563953488372
0.7429748062015504


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [110]:
#p.162
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf),
('svc', svm_clf)],voting='hard')
voting_clf.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',...
                                        

In [111]:
# train performance
y_pred =voting_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred =voting_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.9945494186046512
0.845203488372093


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [112]:
#p.163
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\ensemble\bagging.py:623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [113]:
# train performance
y_pred =bag_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred =bag_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8521681201550387
0.8481104651162791


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [114]:
#p.165
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [115]:
# train performance
y_pred =rnd_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred =rnd_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8630692829457365
0.8551356589147286


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [118]:
#p.170
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
DecisionTreeClassifier(max_depth=2), n_estimators=150,
algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=2,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [119]:
# train performance
y_pred =ada_clf.predict(X_train)
n_correct = sum(y_pred == y_train.as_matrix().ravel())
print(n_correct / len(y_pred))
# test performance
y_pred =ada_clf.predict(X_test)
n_correct = sum(y_pred == y_test.as_matrix().ravel())
print(n_correct / len(y_pred))

0.9114583333333334
0.8866279069767442


C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\shu0503\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
